In [6]:
import pandas as pd
import transformers

Nedávno jsem narazil na [repozitář](https://github.com/UnderstandLingBV/LLaMa2lang), ve kterém autoři tvrdili, že je s pomocí jejich postupu možné finetunovat on-prem large language model (LLM) tak, aby dokázal mluvit prakticky jakýmkoli jazykem. Vzhledem k tomu, že množina LLMek použitelných na lokálu a mluvících česky je extrémně omezená, mne to pochopitelně zaujalo. Nemaje HW ani ochotu zaplatit si kvůli experimentu s nejasným výsledkem cloud jsem se nicméně nezaměřil ma finetuning LLMka, nýbrž embedding modelu z balíčku sentence transformers.  
Bohužel výsledek byl z mnoha důvodů tristní. Jelikož by se mi ale tento kód někdy v budoucnu mohl hodit, umísťuji ho na Github.

## Příprava dat

Nejprve je třeba dostat se k nějakým trénovacím datům. Jednou z možností je použít dataset OpenAssistant/oasst1. Ten se naléza na [Hugging facu](https://huggingface.co/datasets/OpenAssistant/oasst1). Je pod Apache-2.0 licencí a tedy ho lze použít i pro komerční účely. Když se podíváme do popisu, zjistíme, že obsahuje i neanglické texty. Českých je ale bohužel jen 372. To nás ale nemusí trápit - údajně lze získat kvalitní data použitím open source překladače (no, nelze, ale to předbíhám).  
Jak se k datům dostaneme? Jeden způsob jejich nahrání na disk a do paměti spočívá v použití metody *load_dataset* z [hugging-facovského balíčku *datasets*](https://huggingface.co/docs/datasets/index). Sem stačí umístit jméno repozitáře a data se stáhnou a zpracují. Při opakovaném spuštění by se neměla znova stahovat z internetu - jsou zjevně uložena v nějakém cache adresáři.

In [5]:
import datasets
orig_dataset = datasets.load_dataset("OpenAssistant/oasst1")

C:\vs\programovani\python\finetuning\environment\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Vidíme, že se dataset skládá z trénovací a validační části. Lze k nim přistupovat jako ke slovníkům (koneckonců datový typ je Dataset**Dict**), tj. skrze předpis typu orig_dataset["train"].

In [6]:
orig_dataset

DatasetDict({
    train: Dataset({
        features: ['message_id', 'parent_id', 'user_id', 'created_date', 'text', 'role', 'lang', 'review_count', 'review_result', 'deleted', 'rank', 'synthetic', 'model_name', 'detoxify', 'message_tree_id', 'tree_state', 'emojis', 'labels'],
        num_rows: 84437
    })
    validation: Dataset({
        features: ['message_id', 'parent_id', 'user_id', 'created_date', 'text', 'role', 'lang', 'review_count', 'review_result', 'deleted', 'rank', 'synthetic', 'model_name', 'detoxify', 'message_tree_id', 'tree_state', 'emojis', 'labels'],
        num_rows: 4401
    })
})

K jednotlivým záznamům se dostaneme skrze indexaci, tj. předpisem ala

In [7]:
orig_dataset["train"][0]

{'message_id': '6ab24d72-0181-4594-a9cd-deaf170242fb',
 'parent_id': None,
 'user_id': 'c3fe8c76-fc30-4fa7-b7f8-c492f5967d18',
 'created_date': '2023-02-05T14:23:50.983374+00:00',
 'text': 'Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.',
 'role': 'prompter',
 'lang': 'en',
 'review_count': 3,
 'review_result': True,
 'deleted': False,
 'rank': None,
 'synthetic': False,
 'model_name': None,
 'detoxify': {'toxicity': 0.00044308538781479,
  'severe_toxicity': 3.252684837207198e-05,
  'obscene': 0.00023475120542570949,
  'identity_attack': 0.0001416115992469713,
  'insult': 0.00039489680784754455,
  'threat': 4.075629112776369e-05,
  'sexual_explicit': 2.712695459194947e-05},
 'message_tree_id': '6ab24d72-0181-4594-a9cd-deaf170242fb',
 'tree_state': 'ready_for_export',
 'emojis': {'name': ['+1', '_skip_reply', '_skip_ranking'],
  'count': [10

Asi pohodlněji se s daty bude pracovat, když budou v pandím dataframu. Ten vytvoříme provoláním metody *to_pandas* na 

In [9]:
orig_train_frame = orig_dataset["train"].to_pandas()
orig_train_frame.head(3)

,message_id,parent_id,user_id,created_date,text,role,lang,review_count,review_result,deleted,rank,synthetic,model_name,detoxify,message_tree_id,tree_state,emojis,labels
0,6ab24d72-0181-4594-a9cd-deaf170242fb,None,c3fe8c76-fc30-4fa7-b7f8-c492f5967d18,2023-02-05T14:23:50.983374+00:00,Can you write a short introduction about the r...,prompter,en,3,True,False,NaN,False,None,"{'toxicity': 0.00044308538781479, 'severe_toxi...",6ab24d72-0181-4594-a9cd-deaf170242fb,ready_for_export,"{'name': ['+1', '_skip_reply', '_skip_ranking'...","{'name': ['spam', 'lang_mismatch', 'pii', 'not..."
1,c8e83833-ecbc-44fe-b6db-735228c25a1c,6ab24d72-0181-4594-a9cd-deaf170242fb,2c96e467-66f0-4be7-9693-bda51356a424,2023-02-06T13:50:44.657083+00:00,"""Monopsony"" refers to a market structure where...",assistant,en,3,True,False,0.0,False,None,"{'toxicity': 0.00026396565954200923, 'severe_t...",6ab24d72-0181-4594-a9cd-deaf170242fb,ready_for_export,"{'name': ['+1', '_skip_labeling'], 'count': [3...","{'name': ['spam', 'fails_task', 'lang_mismatch..."
2,6708c47f-05c9-4346-b3d2-40b2bd24fde4,c8e83833-ecbc-44fe-b6db-735228c25a1c,2c96e467-66f0-4be7-9693-bda51356a424,2023-02-06T18:48:49.391686+00:00,Now explain it to a dog,prompter,en,3,True,False,NaN,False,None,"{'toxicity': 0.03648477792739868, 'severe_toxi...",6ab24d72-0181-4594-a9cd-deaf170242fb,ready_for_export,None,"{'name': ['spam', 'lang_mismatch', 'pii', 'not..."


Popravdě ale asi jednodušší postup spočívá v tom, že si z repozitáře - konkrétně ze záložky ["Files and versions"](https://huggingface.co/datasets/OpenAssistant/oasst1/tree/main/data), adresáře "data" - stáhneme parquet soubory a ty načteme pandasí funkcí *read_parquet*.

In [11]:
orig_train_frame  =  pd.read_parquet("train-00000-of-00001-b42a775f407cee45.parquet")
orig_train_frame.head(3)

,message_id,parent_id,user_id,created_date,text,role,lang,review_count,review_result,deleted,rank,synthetic,model_name,detoxify,message_tree_id,tree_state,emojis,labels
0,6ab24d72-0181-4594-a9cd-deaf170242fb,None,c3fe8c76-fc30-4fa7-b7f8-c492f5967d18,2023-02-05T14:23:50.983374+00:00,Can you write a short introduction about the r...,prompter,en,3,True,False,NaN,False,None,"{'toxicity': 0.00044308538781479, 'severe_toxi...",6ab24d72-0181-4594-a9cd-deaf170242fb,ready_for_export,"{'name': ['+1', '_skip_reply', '_skip_ranking'...","{'name': ['spam', 'lang_mismatch', 'pii', 'not..."
1,c8e83833-ecbc-44fe-b6db-735228c25a1c,6ab24d72-0181-4594-a9cd-deaf170242fb,2c96e467-66f0-4be7-9693-bda51356a424,2023-02-06T13:50:44.657083+00:00,"""Monopsony"" refers to a market structure where...",assistant,en,3,True,False,0.0,False,None,"{'toxicity': 0.00026396565954200923, 'severe_t...",6ab24d72-0181-4594-a9cd-deaf170242fb,ready_for_export,"{'name': ['+1', '_skip_labeling'], 'count': [3...","{'name': ['spam', 'fails_task', 'lang_mismatch..."
2,6708c47f-05c9-4346-b3d2-40b2bd24fde4,c8e83833-ecbc-44fe-b6db-735228c25a1c,2c96e467-66f0-4be7-9693-bda51356a424,2023-02-06T18:48:49.391686+00:00,Now explain it to a dog,prompter,en,3,True,False,NaN,False,None,"{'toxicity': 0.03648477792739868, 'severe_toxi...",6ab24d72-0181-4594-a9cd-deaf170242fb,ready_for_export,None,"{'name': ['spam', 'lang_mismatch', 'pii', 'not..."


Co přesně se v dataframu nachází? Samotný text se nepřekvapivě nalézá ve sloupci "text". Jsou zde jak otázky, tak odpovědi (přesněji tu najdeme dialogové stromy otázek a odpovědí). A to sice ne pohromadě, ale v separátních řádcích. Kategorii textu udává sloupec "role" - hodnota "prompter" značí otázku, hodnota "assistant" odpověď. Hned vedle vidíme i sloupec "lang" signalizující jazyk textu. Vidíme, že je zde krom angličtiny i hromada španělských textů. Českých řádků tu máme pouhých 12. Zajímavá je skutečnost, že počty textů jsou pro (asi) všechny jazyky menší než bylo uvedeno v kartě datasetu na HF...

In [18]:
orig_train_frame["lang"].value_counts()

lang
en       39283
es       22763
ru        7242
zh        3314
de        3050
fr        2474
th        1460
pt-BR     1165
ca        1158
uk-UA      587
it         554
ja         363
pl         304
eu         250
vi         191
hu          75
ar          56
da          44
tr          37
ko          24
fi          18
id          12
cs          12
sv           1
Name: count, dtype: int64

In [20]:
orig_train_frame[orig_train_frame["lang"]=="cs"].head(3)

,message_id,parent_id,user_id,created_date,text,role,lang,review_count,review_result,deleted,rank,synthetic,model_name,detoxify,message_tree_id,tree_state,emojis,labels
54068,bc35002a-5a15-443e-a710-65bdf6180df8,None,cd4e0925-f3f0-478e-b6d5-a9fd66a614cb,2023-02-14T23:06:37.401747+00:00,Jaké je hlavní město Japonska?,prompter,cs,3,True,False,NaN,False,None,None,bc35002a-5a15-443e-a710-65bdf6180df8,ready_for_export,None,"{'name': ['spam', 'lang_mismatch', 'pii', 'not..."
54069,9cb061fb-a49a-44f6-8c07-3ce7e607d295,bc35002a-5a15-443e-a710-65bdf6180df8,7039e007-a81e-4c7a-854e-8e96b3733274,2023-02-16T13:26:05.822985+00:00,Hlavním městem Japonska je Tokio (japonsky 東京)...,assistant,cs,3,True,False,0.0,False,None,None,bc35002a-5a15-443e-a710-65bdf6180df8,ready_for_export,"{'name': ['+1'], 'count': [2]}","{'name': ['spam', 'fails_task', 'lang_mismatch..."
54070,34046cbb-49ca-4988-883a-4c1c59681bf2,9cb061fb-a49a-44f6-8c07-3ce7e607d295,c1a70c09-5b67-4c6d-95d6-866aff79e159,2023-02-21T15:06:36.085459+00:00,Jaké je hlavní město České republiky?,prompter,cs,3,True,False,NaN,False,None,None,bc35002a-5a15-443e-a710-65bdf6180df8,ready_for_export,"{'name': ['+1'], 'count': [1]}","{'name': ['spam', 'lang_mismatch', 'pii', 'not..."


Pro jednoduchost vyhodíme všechny neanglické dokumenty.

In [21]:
orig_train_frame = orig_train_frame[orig_train_frame["lang"]=="en"]
len(orig_train_frame)

39283

Výše bylo uvedeno, že otázky a odpovědi jsou v různých řádcích. Jak je spojit dohromady? Identifikátor záznamu se nachází ve sloupci "message_id". Pakliže se jedná o odpověď, nalezneme v sloupci "parent_id" identifikátor (message_id) otázky. Otázky mají v parent_id hodnotu None (pokud šlo o otázku uvozující celou konverzaci) anebo message_id odpovědi na předchozí otázku.  
Všimněme si, že odpovědí na jednu otázku může být více. Odpovědi podle vhodnosti seřadil nějaký člověk (viz [tady](https://arxiv.org/pdf/2304.07327.pdf) a [tady](https://projects.laion.ai/Open-Assistant/docs/tasks/rank_assistant_replies)), přičemž menší číslo je lepší a začíná se nulou.

In [29]:
orig_train_frame[orig_train_frame["parent_id"]=="6ab24d72-0181-4594-a9cd-deaf170242fb"]

,message_id,parent_id,user_id,created_date,text,role,lang,review_count,review_result,deleted,rank,synthetic,model_name,detoxify,message_tree_id,tree_state,emojis,labels
1,c8e83833-ecbc-44fe-b6db-735228c25a1c,6ab24d72-0181-4594-a9cd-deaf170242fb,2c96e467-66f0-4be7-9693-bda51356a424,2023-02-06T13:50:44.657083+00:00,"""Monopsony"" refers to a market structure where...",assistant,en,3,True,False,0.0,False,None,"{'toxicity': 0.00026396565954200923, 'severe_t...",6ab24d72-0181-4594-a9cd-deaf170242fb,ready_for_export,"{'name': ['+1', '_skip_labeling'], 'count': [3...","{'name': ['spam', 'fails_task', 'lang_mismatch..."
3,343ee2d4-87ae-41fd-a768-bdd65959dc4a,6ab24d72-0181-4594-a9cd-deaf170242fb,49ddcb0d-6588-43bd-858d-19c577f12e7b,2023-02-06T13:37:56.044680+00:00,Monopsony is a market structure in which there...,assistant,en,3,True,False,1.0,False,None,"{'toxicity': 0.0008866374846547842, 'severe_to...",6ab24d72-0181-4594-a9cd-deaf170242fb,ready_for_export,"{'name': ['+1', '_skip_reply', '_skip_labeling...","{'name': ['spam', 'fails_task', 'lang_mismatch..."
5,636dd191-50df-4894-ba9a-cd7f00767258,6ab24d72-0181-4594-a9cd-deaf170242fb,c212120c-0b79-4b32-a5b1-d96686ca92ca,2023-02-06T14:28:09.611612+00:00,Monopsony refers to a market structure where t...,assistant,en,3,True,False,2.0,False,None,"{'toxicity': 0.0002960403508041054, 'severe_to...",6ab24d72-0181-4594-a9cd-deaf170242fb,ready_for_export,"{'name': ['+1', '_skip_reply'], 'count': [6, 3]}","{'name': ['spam', 'fails_task', 'lang_mismatch..."


Ideálně tedy chceme dataframe, kde bude na jednom řádku otázka a nejvhodnější odpověď.

In [37]:
questions_frame = orig_train_frame[orig_train_frame["role"]=="prompter"][["message_id", "text"]]
best_answers_frame = orig_train_frame[
                         (orig_train_frame["role"]=="assistant")
                         & (orig_train_frame["rank"]==0.0)
                    ][["parent_id", "text"]]
qa_frame = questions_frame.merge(best_answers_frame, how="inner", left_on="message_id", right_on="parent_id")
qa_frame = qa_frame[["text_x", "text_y"]]
qa_frame.columns = ["question", "answer"]
qa_frame.head(3)

,question,answer
0,Can you write a short introduction about the r...,"""Monopsony"" refers to a market structure where..."
1,What can be done at a regulatory level to ensu...,Here are some potential regulatory options to ...
2,Can you explain contrastive learning in machin...,Sure! Let's say you want to build a model whic...


Jelikož překlad budeme asi realizovat na Google Colabu, uložíme si dataframe do souboru, který na GC později nahrajeme.

qa_frame.to_csv("qa_frame_en.csv", sep="|", index=False)

## Překlad

Pro překlad použijeme model od University of Helsinky, který je k dispozici na jejich repozitáři [na Hugging facu](https://huggingface.co/Helsinki-NLP). Nicméně který z hromady Opus modelů máme vlastně vybrat? Je nutné vědět, že tyto slovníkové modely jsou jen jednosměrné, přičemž směr lze rozklíčovat z názvu podle předpisu opus-mt-{zdrojový_jazyk}-{cílový_jazyk}. Tj. pro překlad z angličtiny do češtiny použijeme "opus-mt-en-cs".  
Načtení modelu realizujeme funkcí *from_pretrained*, do které vložíme buďto repozitář modelu, anebo adresář na disku, do kterého jsme model stáhli. V příkladu je uvedený adresář na disku; pokud bychom chtěli napojení přímo na repozitář, bylo by v kulatých závorkách "Helsinki-NLP/opus-mt-en-cs". Funkci musíme použít nejen na čistě "modelové" transformers.AutoModelForSeq2SeqLM, ale i na *transformers.AutoTokenizer* kvůli převodu slov coby posloupností znaků na slovníková čísla slova zastupující.

In [39]:
import transformers
tokenizer = transformers.AutoTokenizer.from_pretrained("helsinky_model_en_cs")
translation_model = transformers.AutoModelForSeq2SeqLM.from_pretrained("helsinky_model_en_cs")

C:\vs\programovani\python\finetuning\environment\Lib\site-packages\transformers\models\marian\tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Jak model použít? Nejprve s pomoci *tokenizer.encode* převedeme anglická slova na posloupnost čísel - idček jednotlivých anglických slov. Následně skrze *translation_model.generate* vyrobíme posloupnost idček slov překladu. Nakonec s *tokenizer.decode* převedeme idčka na česká slova.  
V *generate* metodě bohužel musíme nastavit *max_length* na 512. Pokud bychom to neudělali, dříve nebo později bychom narazili na errorovou hlášku
```
Token indices sequence length is longer than the specified maximum sequence length for this model (524 > 512). Running this sequence through the model will result in indexing errors
```
Pozn.: %%time tu mám jen pro odhad, jak asi dlouho bude trvat překlad celého dataframu.

In [45]:
%%time
test_text = 'Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.'
translation_inputs = tokenizer.encode(test_text, return_tensors="pt")
translated_tokens = translation_model.generate(translation_inputs, max_length=512)
tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

CPU times: total: 5.42 s
Wall time: 1.38 s


'Můžete napsat krátký úvod o významu pojmu "monopsonie" v ekonomice? Prosím použijte příklady týkající se potenciálních monopsonií na trhu práce a citujte relevantní výzkum.'

In [61]:
def translate_text(tokenizer, translation_model, source_text:str)->str:
    """Translate text from one language to other.

    Args:
        source_text(str): Text in original language.

    Return:
        str: Text in wanted language.
    """
    translation_inputs = tokenizer.encode(source_text, return_tensors="pt")
    translated_tokens = translation_model.generate(translation_inputs, max_length=512)
    return tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

V následující buňce je realizován překlad. Překlad by na mém počítači trval několik hodin za maximálního vytížení CPU. Bude tedy lepší ho pustit na Colabu.  
Pozn.: občas nějaký překlad selže na jakémsi problému s indexováním. Jelikož nepotřebujeme přeložit každý jednotlivý pár otázka-odpověď, řešíme to try-except konstrukcí.

In [66]:
%%time
translated_questions_list = []
translated_aswers_list = []

starting_index = 0
ending_index = 3

sub_qa_frame = qa_frame[
                   qa_frame.index.isin(
                       range(starting_index, ending_index)
                   )
               ]

for one_row in sub_qa_frame.iterrows():
    row_index = one_row[0]
    if row_index%2 == 0:
        print(f"Processing row {row_index}")
    one_question = one_row[1]["question"]
    one_answer = one_row[1]["answer"]
    try:
        translated_question = translate_text(tokenizer, translation_model, one_question) 
        translated_answer = translate_text(tokenizer, translation_model, one_answer)
        translated_questions_list.append(translated_question)
        translated_aswers_list.append(translated_answer)
    except:
        translated_questions_list.append("error")
        translated_aswers_list.append("error")

Processing row 0
Processing row 2
CPU times: total: 1min 39s
Wall time: 25.3 s


Níže je vidět, že překlad funguje, ale ideální zrovna není. Možná to bude stačit pro embeddingový model, ale u LLM bych měl trochu obavy.

In [67]:
translated_questions_list

['Můžete napsat krátký úvod o významu pojmu "monopsonie" v ekonomice? Prosím použijte příklady týkající se potenciálních monopsonií na trhu práce a citujte relevantní výzkum.',
 'Co lze udělat na regulační úrovni, aby se v monopsonii zajistilo, že moc nad zaměstnanci není zneužívána? Seznam několika možností a zaměřit se na orgány, které by měly jednat.',
 'Můžete vysvětlit kontrastní učení ve strojovém učení jednoduchým způsobem pro někoho nového v oblasti ML?']

In [68]:
translated_aswers_list

['"Monopsony" odkazuje na tržní strukturu, kde je pouze jeden kupující pro konkrétní dobro nebo službu. V ekonomice, tento pojem je zvláště relevantní na trhu práce, kde monopsony zaměstnavatel má významnou moc nad mzdy a pracovní podmínky svých zaměstnanců. Přítomnost monopsonie může vést k nižší mzdy a snížené pracovní příležitosti pro pracovníky, protože zaměstnavatel má jen malou motivaci ke zvýšení mezd nebo k poskytování lepších pracovních podmínek. Nedávný výzkum identifikoval potenciální monopsonies v odvětvích, jako je maloobchod a rychlé občerstvení, kde několik velkých společností kontroluje významnou část trhu (Bivens & Mishel, 2013). V těchto odvětvích, pracovníci často čelí nízké mzdy, omezené výhody, a snížení vyjednávací síly, což vede k situaci, kdy jsou závislí na zaměstnavateli pro jejich živobytí. Tato závislost může vést k dalšímu potlačení mezd a poklesu pracovních podmínek. Celkově, koncept monopsonie je nezbytný pro pochopení dynamiky trhu práce a vlivu tržní sí

OK, máme kód a datové soubory a rádi bychom tyto věci dostali na Google Colab. Bohužel uploadování souborů o velikosti malých jednotek MB skrz GUI, když člověk používá Firefox, zjevně možné není. Pokus o takovou aktivitu totiž skončí červeným kolečkem (asi timeout?). Musí se na to jinak - přes kód. Konkrétně přes funkci *files.upload* z *google.colab*.
```python
from google.colab import files
uploaded_files_dict = files.upload()
```
Po jejím spuštění se objeví uploadovací tlačítko, skrz které lze uploadovaný soubor vybrat. Navíc člověk vidí, na kolik procent je v daném okamžiku soubor nahrán.  
Důležité je poznamentat, že *upload* funkce nevrací soubor, nýbrž slovník. Zde klíčem není jméno souboru na lokálu, ale na cloudu. Tj. když si nejsme jistí, raději použijeme *uploaded_files_dict.keys()*. Věc ve slovníku navíc není soubor, ale posloupnost bytů (aka objekt typu "bytes"). Asi nejrozumnější bude tuto věc napřed uložit do souboru a pak se souborem již pracovat normálně. Tj. použijeme kód
```python
uploaded_file_content = uploaded_files_dict["qa_frame_en.csv"]
with open("/home/qa_frame_en.csv", 'wb') as some_file:
    some_file.write(uploaded_file_content)
```
Následně uložený soubor otevřeme pomocí
```python
qa_frame = pd.read_csv("/home/qa_frame_en.csv", sep="|")
qa_frame.head(3)
```
Balíček transformers je sice již na Colabu předinstalovaný, při snaze načíst model a tokenizér ale bohužel obdržíme chybovou hlášku
```
ValueError: This tokenizer cannot be instantiated. Please make sure you have `sentencepiece` installed in order to use this tokenizer.
```
Takže instalaci balíčků se nevyhneme - musíme do buňky napsat
```
!pip install sentencepiece
```
A asi bude třeba i restartovat kernel. To se v Colabu provede kliknutím na ikonu "Command palete" (takový "vizitkový" obedélníček ve spodku panelu u levého okraje obrazovky) a napsáním "restart session" do objevivší se nabídky. Následně lze vložit a pustit výše uvedený kód.  
Rychlost tedy není nic moc - tam, kde byl na lokále wall time 25s, je na Colabu wall time 51s. Možná pomůže přesun modelu na GPU. Nejprve se musíme z CPU modu přepnout do jednoho z modů využívajících grafické karty. To provedeme kliknutím na "Runtime" (nabídka nahoře) -> "Change runtime type" a zvolením T4GPU. To, že systém GPU vidí, ověříme pomocí *torch.cuda.is_available()*. Následně použijeme prakticky stejný kód jako předtím, pouze přidáme řádek
```python
device = torch.device("cuda")
```
a na konec relevatních řádků přidáme ".to(device)". Relevantními řádky máme na mysli
```python
translation_model = transformers.AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-cs").to(device)
```
a
```python
translation_inputs = tokenizer.encode(source_text, return_tensors="pt").to(device)
```
Tj. celý kód má podobu (co zde jeden segment, to buňka v Colabu):
```
!pip install sentencepiece
```

```python
import pandas as pd
import torch
import transformers
from google.colab import files

device = torch.device("cuda")
uploaded_files_dict = files.upload()
uploaded_file_content = uploaded_files_dict["qa_frame_en.csv"]

with open("/home/qa_frame_en.csv", 'wb') as some_file:
    some_file.write(uploaded_file_content)

qa_frame_en = pd.read_csv("/home/qa_frame_en.csv", sep="|")
qa_frame_en.head(3)
```

```python
def translate_text(tokenizer, translation_model, source_text:str, device)->str:
    """Translate text from one language to other.

    Args:
        source_text(str): Text in original language.

    Return:
        str: Text in wanted language.
    """
    translation_inputs = tokenizer.encode(source_text, return_tensors="pt").to(device)
    translated_tokens = translation_model.generate(translation_inputs, max_length=512)
    return tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

tokenizer = transformers.AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-cs")
translation_model = transformers.AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-cs").to(device)
```

```python
%%time
translated_questions_list = []
translated_answers_list = []

starting_index = 0
ending_index = 100

sub_qa_frame_en = qa_frame_en[
                   qa_frame_en.index.isin(
                       range(starting_index, ending_index)
                   )
               ]

for one_row in sub_qa_frame_en.iterrows():
    row_index = one_row[0]
    if row_index%2 == 0:
        print(f"Processing row {row_index}")
    one_question = one_row[1]["question"]
    one_answer = one_row[1]["answer"]
    try:
        translated_question = translate_text(tokenizer, translation_model, one_question, device)
        translated_answer = translate_text(tokenizer, translation_model, one_answer, device)
        translated_questions_list.append(translated_question)
        translated_answers_list.append(translated_answer)
    except Exception as problem:
        print(f"Error message: {problem}")
        translated_questions_list.append("error")
        translated_aswers_list.append("error")
```
Rychlost je mnohem větší - tedy dokud nedojdou resourcy (přidělený čas na GPU)...
Každopádně nakonec se musí vygenerované překlady uložit do csv:
```python
pd.DataFrame({
    "question": translated_questions_list,
    "answer": translated_aswers_list
}).to_csv(f"/home/qa_frame_cs_{starting_index}_{ending_index}.csv", sep="|", index=False)
```

## Pokus o finetunování embedding modelu
Nakonec jsem nechal přeložit jen 500 párů otázka-odpověď. NIcméně byl jsem zvědavý, jestli to vůbec něco s embeddingovým modelem udělá. Při trénování jsem vycházel z [tohoto](https://huggingface.co/blog/how-to-train-sentence-transformers) materiálu. 

In [8]:
from sentence_transformers import SentenceTransformer, util
from datasets import load_dataset
from sentence_transformers import losses
from torch.utils.data import DataLoader
from sentence_transformers import InputExample

c:\primary\programovani\python\finetuning\environment\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Napřed si stáhneme/načteme embeddingový model.

In [9]:
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

Následně si ho ještě před přetrénováním vyzkoušíme.

In [3]:
query_embedding_a = model.encode('Kolik stojí debentí karta?')
doc_embedding_b = model.encode('Debentní karta stojí 100 Kč.')
doc_embedding_c = model.encode('Karlova univerzita byla založena roku 1367.')

In [4]:
print(util.cos_sim(query_embedding_a, doc_embedding_b))
print(util.cos_sim(query_embedding_a, doc_embedding_c))

tensor([[0.7244]])
tensor([[-0.0681]])


In [5]:
kb_doc_q = model.encode('Nepovedlo se otevřít aplikaci KB Podpisový modul.')
kb_doc_a = model.encode("Pokud se Vám během přihlášení pomocí 'Osobní certifikát na čipové kartě'  zobrazilo jedno z následujících hlášení, doporučujeme nainstalovat / aktualizovat náš KB Podpisový modul. Je možné, že ho nemáte nainstalovaný a nebo máte starší verzi aplikace. Pro Windows 8.1 a výše můžete použít tento PDF návod, pro případný Windows 7 máme dočasné řešení ZDE.")
util.cos_sim(kb_doc_q, kb_doc_a)

tensor([[0.5427]])

Pro finetunovací data bude načítací funkce očekávat, že dostane jméno adresáře obsahující jsonl soubory. To jsou soubory, které mají na každém řádky jeden json. V našem konkrétním případě bude klíčem "set" a hodnotou list s otázkou a odpovědí. Tj. obsah souboru bude vypadat nějak takto:
```
{"set": ["Zde bude první otázka.", "Zde bude odpověď na první otázku."]}
{"set": ["Zde bude druhá otázka.", "Zde bude odpověď na druhou otázku."]}
```
Jak ale naše csv na jsonl převedeme?  
Napřed si vyhodíme errorové záznamy.

In [7]:
qa_from_csv = pd.read_csv("qa_frame_cs_0_500.csv", sep="|")
print(len(qa_from_csv))
qa_from_csv = qa_from_csv[qa_from_csv["answer"]!= "error"]
print(len(qa_from_csv))

500
472


Dataframe si převedem na slovníky a ty potom jeden po druhém zapíšeme do souboru. Parametr *ensure_ascii* ve funkci *json.dumps* zajistí, že české znaky nebudou escapovány.

In [27]:
import json

qa_to_jsonl = qa_from_csv.to_dict(orient="records")
with open("cs_finetun_data\\qa_cs_0_500.jsonl", "w", encoding="utf8") as output_file:
    for one_dict in qa_to_jsonl:
        edited_dict = {"set": [one_dict["question"], one_dict["answer"]]}
        output_file.write(json.dumps(edited_dict, ensure_ascii=False) + "\n")

Dataset pak načteme funkcí *load_dataset* z balíčku *datasets*. Všimněme si, že jako parametr neuvádíme jméno jsonl souboru, nýbrž adresář, kde se onen soubor nachází.

In [6]:
dataset = load_dataset("cs_finetun_data")

Generating train split: 472 examples [00:00, 85148.88 examples/s]


In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['set'],
        num_rows: 472
    })
})

Dataset musíme převést na list objektů InputExample, které jsou fitovací funkcí (resp. DataLoaderem) očekávané.

In [32]:
train_examples = []
for one_example in dataset["train"]["set"]:
    train_examples.append(
        InputExample(texts=[one_example[0], one_example[1]])
    )

In [33]:
train_loss = losses.MultipleNegativesRankingLoss(model=model)

In [34]:
train_dataloader = DataLoader(train_examples, shuffle=False, batch_size=4)

In [40]:
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=5) 

Epoch: 100%|██████████| 5/5 [01:10<00:00, 14.16s/it]


A jaký je výsledek trénování? Nic se nezměnilo (resp. výsledky jsou mírně horší).

In [41]:
query_embedding_a = model.encode('Kolik stojí debentí karta?')
doc_embedding_b = model.encode('Debentní karta stojí 100 Kč.')
doc_embedding_c = model.encode('Karlova univerzita byla založena roku 1367.')

In [42]:
print(util.cos_sim(query_embedding_a, doc_embedding_b))
print(util.cos_sim(query_embedding_a, doc_embedding_c))

tensor([[0.7244]])
tensor([[-0.0681]])


In [43]:
kb_doc_q = model.encode('Nepovedlo se otevřít aplikaci KB Podpisový modul.')
kb_doc_a = model.encode("Pokud se Vám během přihlášení pomocí 'Osobní certifikát na čipové kartě'  zobrazilo jedno z následujících hlášení, doporučujeme nainstalovat / aktualizovat náš KB Podpisový modul. Je možné, že ho nemáte nainstalovaný a nebo máte starší verzi aplikace. Pro Windows 8.1 a výše můžete použít tento PDF návod, pro případný Windows 7 máme dočasné řešení ZDE.")
util.cos_sim(kb_doc_q, kb_doc_a)

tensor([[0.5427]])

Zkusme ale vzít česká data z [OpenOrca datasetu](https://huggingface.co/datasets/Open-Orca/OpenOrca/tree/main). Přesněji tedy z jeho menší části.

In [ ]:
par_df = pd.read_parquet("1M-GPT4-Augmented.parquet")
par_df.head(3)

,id,system_prompt,question,response
0,niv.242684,,You will be given a definition of a task first...,"[\n [""AFC Ajax (amateurs)"", ""has ground"", ""Sp..."
1,flan.564327,You are an AI assistant. You will be given a t...,Generate an approximately fifteen-word sentenc...,Midsummer House is a moderately priced Chinese...
2,flan.1875913,"You are a helpful assistant, who always provid...",What happens next in this paragraph?\n\nShe th...,C. She then dips the needle in ink and using t...


Dataset obsahuje většinově nečeské záznamy. Jak je odstraníme? Použijeme balíček langdetect.

In [ ]:
from langdetect import detect

def is_czech(sentence):
    try:
        return detect(sentence) == 'cs'
    except:
        return False


A pak hodně pomalé apply (kontrola trvala déle než půl hodiny).

In [ ]:
par_df['is_czech'] = par_df['question'].apply(is_czech)

Českých záznamů tu opravdu mnoho není.

In [ ]:
len(par_df)

994896

In [ ]:
len(par_df[par_df["is_czech"]==True])

2396

A obvykle se navíc jedná o překlady či detekci jazyka od GPT modelu.

In [ ]:
par_df[par_df["is_czech"]==True].head(3)

,id,system_prompt,question,response,is_czech
155,flan.2379918,You are an AI assistant. User will you give yo...,"Tato situace má celou řadu příčin, které jsou ...",Step 1: Determine the source language - The te...,True
716,niv.105996,You are an AI assistant. Provide a detailed an...,"In this task, you need to Translate Czech text...",The solution concerns the suspension of drive ...,True
761,flan.1459354,"You are a helpful assistant, who always provid...","c) údaje ke spisům týkajícím se vyšetřování, k...",c) Information about files related to investig...,True


In [ ]:
czech_parquet_frame = par_df[par_df["is_czech"]==True]

A celou operaci si zopakujeme (s tím, že je vhodné udělat reload původního modelu, abychem nepřetrénovávali už přetrénovaný model).

In [ ]:
import json

qa_to_jsonl = czech_parquet_frame[["question", "response"]].to_dict(orient="records")
with open("cs_finetun_data_2\\qa_parquest_data.jsonl", "w", encoding="utf8") as output_file:
    for one_dict in qa_to_jsonl:
        edited_dict = {"set": [one_dict["question"], one_dict["response"]]}
        output_file.write(json.dumps(edited_dict, ensure_ascii=False) + "\n")

In [ ]:
dataset = load_dataset("cs_finetun_data_2")

Generating train split: 2396 examples [00:00, 152483.12 examples/s]


In [ ]:
train_examples = []
for one_example in dataset["train"]["set"]:
    train_examples.append(
        InputExample(texts=[one_example[0], one_example[1]])
    )

In [ ]:
train_loss = losses.MultipleNegativesRankingLoss(model=model)

In [ ]:
train_dataloader = DataLoader(train_examples, shuffle=False, batch_size=4)

In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=5) 

Epoch: 100%|██████████| 5/5 [06:12<00:00, 74.60s/it]


Výsledek? Je to ještě horší než předtím :D.

In [ ]:
kb_doc_q = model.encode('Nepovedlo se otevřít aplikaci KB Podpisový modul.')
kb_doc_a = model.encode("Pokud se Vám během přihlášení pomocí 'Osobní certifikát na čipové kartě'  zobrazilo jedno z následujících hlášení, doporučujeme nainstalovat / aktualizovat náš KB Podpisový modul. Je možné, že ho nemáte nainstalovaný a nebo máte starší verzi aplikace. Pro Windows 8.1 a výše můžete použít tento PDF návod, pro případný Windows 7 máme dočasné řešení ZDE.")
util.cos_sim(kb_doc_q, kb_doc_a)

tensor([[0.4661]])

In [ ]:
query_embedding_a = model.encode('Kolik stojí debentí karta?')
doc_embedding_b = model.encode('Debentní karta stojí 100 Kč.')
doc_embedding_c = model.encode('Karlova univerzita byla založena roku 1367.')

In [ ]:
print(util.cos_sim(query_embedding_a, doc_embedding_b))
print(util.cos_sim(query_embedding_a, doc_embedding_c))

tensor([[0.6777]])
tensor([[-0.1093]])
